# Yolov5 high resolution training

In [ ]:
VER = 'vclbyolo5_34'
WORK_DIR = '/content/drive/MyDrive/reef'

In [ ]:
import os
import cv2
import json
import random
import torch
import numpy as np
import pandas as pd
from shutil import copyfile
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
CONFIG = {
    'ver': VER,
    'bbone': 'yolov5s6.pt', # yolov5s6.pt yolov5m6.pt
    'width': 1280, 
    'height': 720,
    'resize': 3840, # 3520 3008
    'batch_size': 2,
    'workers': 4,
    'bal_split': 'train_split_balanced_v1', #'train_split_balanced_v1', None
    'val_fold': 4,
    'val_video': 2,
    'empty_sh': .2,
    'mixup_prob': .5,
    'scale': .5, # default .5
    'shear': .2, # default 0
    'flipud': .5, # default .5
    'fliplr': .5, # default .5
    'epochs': 16,
    'seed': 2022
}
DATA_PATH = f'{WORK_DIR}/data'
if CONFIG["bal_split"]:
    YDATA_PATH = f'{WORK_DIR}/data_vv5_bsvf{CONFIG["val_fold"]}'
else:
    YDATA_PATH = f'{WORK_DIR}/data_vv5_{CONFIG["val_video"]}'
MDLS_PATH = f'{WORK_DIR}/models_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/config.json', 'w') as file:
    json.dump(CONFIG, file)

def seed_all(seed=0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state    
    
random_state = seed_all(CONFIG['seed'])

## Images and annotations

In [ ]:
train = pd.read_csv(f'{DATA_PATH}/{CONFIG["bal_split"]}.csv') if CONFIG["bal_split"] else pd.read_csv(f'{DATA_PATH}/train.csv') 
print(train.shape)
train['pos'] = train.annotations != '[]'
print(train.shape)
train.head()

In [ ]:
fold = CONFIG["val_fold"] if CONFIG["bal_split"] else CONFIG["val_video"]
if not os.path.exists(YDATA_PATH):
    os.mkdir(YDATA_PATH)
    for typ in ['images', 'labels']:
        path = f'{YDATA_PATH}/{typ}'
        if not os.path.exists(path):
            os.mkdir(path)
        for mode in ['train', 'val']:
            path = f'{YDATA_PATH}/{typ}/{mode}'
            if not os.path.exists(path):
                os.mkdir(path)
    annos = []
    for i, x in tqdm(train.iterrows(), total=len(train)):
        if CONFIG["bal_split"]:
            if x.fold_id == fold:
                mode = 'val'
            else:
                mode = 'train'
                if not x.pos: continue
        else:
            if x.video_id == fold:
                mode = 'val'
            else:
                mode = 'train'
                if not x.pos: continue
        copyfile(
            f'{DATA_PATH}/train_images/video_{x.video_id}/{x.video_frame}.jpg',
            f'{YDATA_PATH}/images/{mode}/{x.image_id}.jpg'
        )
        if not x.pos:
            continue
        r = ''
        anno = eval(x.annotations)
        for an in anno:
            r += '0 {} {} {} {}\n'.format(
                (an['x'] + an['width'] / 2) / CONFIG['width'],
                (an['y'] + an['height'] / 2) / CONFIG['height'],
                an['width'] / CONFIG['width'], 
                an['height'] / CONFIG['height']
            )
        with open(f'{YDATA_PATH}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
            fp.write(r)

## Train

In [ ]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: <scale>  # image scale (+/- gain)
shear: <shear>  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: <flipud>  # image flip up-down (probability)
fliplr: <fliplr>  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''
hyps = hyps.replace('<scale>', str(CONFIG['scale']))
hyps = hyps.replace('<shear>', str(CONFIG['shear']))
hyps = hyps.replace('<flipud>', str(CONFIG['flipud']))
hyps = hyps.replace('<fliplr>', str(CONFIG['fliplr']))
print(hyps)

In [ ]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: <path> #../yolo_data/fold1/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['star']  # class names

# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''
data = data.replace('<path>', YDATA_PATH)
print(data)

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
with open('./data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [ ]:
!ls data/

In [ ]:
!ls -la -r runs/train/

In [ ]:
model_name = f'model_{CONFIG["ver"]}'
resize = CONFIG['resize']
batch_size = CONFIG['batch_size']
epochs = CONFIG['epochs']
bbone = CONFIG['bbone']

In [ ]:
!python train.py \
    --img $resize \
    --batch $batch_size \
    --epochs $epochs \
    --data reef_f1_naive.yaml \
    --weights $bbone \
    --name $model_name \
    --hyp data/hyps/hyp.heavy.2.yaml

In [ ]:
!cp -r runs/train/$model_name/* $MDLS_PATH/

## Inference

In [ ]:
model = torch.hub.load(
    '.', 
    'custom', 
    path=f'{MDLS_PATH}/weights/best.pt',
    source='local',
    force_reload=True
)
model.conf = 0.01

In [ ]:
def draw_boxes(img, bboxes, scores=None):
    color = (0, 255, 0) if scores else (0, 0, 255)
    for i in range(len(bboxes)):
        box = bboxes[i]
        text = '{} {:.1f}%'.format('pred', scores[i] * 100) if scores else 'gt'
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = int(box[2])
        y1 = int(box[3])
        cv2.rectangle(img, (x0, y0), (x1, y1), color, 2)
        cv2.putText(
            img, 
            text, 
            (x0, y0 - 3), 
            cv2.FONT_HERSHEY_PLAIN, 
            1.4, 
            color, 
            thickness=2
        )
    return img

In [ ]:
%matplotlib inline

count = 0
max_count = 8

for i, x in train.iterrows():
    #if x.video_id == fold:
    if x.fold_id == fold:
        if not x.pos:
            continue
        count += 1
        img_path = f'{YDATA_PATH}/images/val/{x.image_id}.jpg'
        img = cv2.imread(img_path)
        print(count, img_path)
        anno = eval(x.annotations)
        gt_bboxes = []
        for an in anno:
            gt_bboxes.append([
                an['x'],
                an['y'],
                an['x'] + an['width'], 
                an['y'] + an['height']               
            ])
        res = model(img, size=CONFIG['resize'], augment=True)
        pred_bboxes = []
        pred_scores = []
        if res.pandas().xyxy[0].shape[0] == 0:
            pass
        else:
            for idx, row in res.pandas().xyxy[0].iterrows():
                pred_bboxes.append([
                    row.xmin, 
                    row.ymin, 
                    row.xmax, 
                    row.ymax                    
                ])
                pred_scores.append(row.confidence)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = draw_boxes(img, gt_bboxes, scores=None)
        img = draw_boxes(img, pred_bboxes, scores=pred_scores)
        plt.figure(figsize=(20, 10))
        plt.imshow(img)
        plt.show()
        if count >= max_count:
            break